# Truss TensorFlow example

<a href="https://colab.research.google.com/github/basetenlabs/truss/blob/main/docs/notebooks/tensorflow_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade tensorflow truss requests
# For help installing tensorflow, see https://www.tensorflow.org/install/pip


In [ ]:
import tensorflow as tf

#Creates tensorflow model
model = tf.keras.applications.ResNet50V2(
    include_top=True,
    weights="imagenet",
    classifier_activation="softmax",
)

In [ ]:
from truss import create

# Create the Truss (serializing & packaging model)
tr = create(model, target_directory="tensorflow_truss")

In [ ]:
#Preprocess and Postprocess Functions
import requests
import tempfile
import numpy as np

from scipy.special import softmax

def preprocess(url):
    """Preprocess step for ResNet"""
    request = requests.get(url)
    with tempfile.NamedTemporaryFile() as f:
        f.write(request.content)
        f.seek(0)
        input_image = tf.image.decode_png(tf.io.read_file(f.name))
    preprocessed_image = tf.keras.applications.resnet_v2.preprocess_input(
        tf.image.resize([input_image], (224, 224))
    )
    return np.array(preprocessed_image)

def postprocess(predictions, k=5):
    """Post process step for ResNet"""
    class_predictions = predictions[0]
    LABELS = requests.get(
        'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt'
    ).text.split('\n')
    class_probabilities = softmax(class_predictions)
    top_probability_indices = class_probabilities.argsort()[::-1][:k].tolist()
    return {LABELS[index]: 100 * class_probabilities[index].round(3) for index in top_probability_indices}

In [ ]:
image = preprocess("https://github.com/pytorch/hub/raw/master/images/dog.jpg")
results = tr.predict(image)
postprocess(results["predictions"])